In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from scipy.sparse import random as sparse_random
from scipy.sparse.linalg import cg  # Conjugate Gradient solver
import time

In [2]:
# Set matrix dimensions and sparsity
matrix_size = 1000  # You can scale this as needed
density = 0.01  # Sparsity of the matrix

# Generate a large random sparse matrix
A = sparse_random(matrix_size, matrix_size, density=density, format='csr', dtype=np.float32)
A = A @ A.T  # Make it symmetric positive-definite
b = np.random.rand(matrix_size).astype(np.float32)  # Random vector for Ax = b

# Solve with traditional PCG for comparison
x_pcg, exit_code = cg(A, b)


In [3]:
# Define the neural network model using TensorFlow/Keras
def build_surrogate_model(input_shape):
    model = tf.keras.Sequential([
        layers.Input(shape=input_shape),  # Updated to use 'shape' instead of 'input_shape'
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(matrix_size)  # Output layer: approximate solution vector x
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build the surrogate model
input_shape = (matrix_size,)  # Each input vector b has size 'matrix_size'
model = build_surrogate_model(input_shape)


In [4]:
# Generate training data
def generate_training_data(num_samples):
    b_vectors = []
    x_solutions = []

    for _ in range(num_samples):
        # Generate synthetic data
        A = sparse_random(matrix_size, matrix_size, density=density, format='csr', dtype=np.float32)
        A = A @ A.T  # Make it symmetric positive-definite
        b = np.random.rand(matrix_size).astype(np.float32)

        # Solve using the PCG solver to get ground truth solution
        x, _ = cg(A, b)  # Ground truth solution

        b_vectors.append(b)  # Input to the model
        x_solutions.append(x)  # Expected output (solution)

    return np.array(b_vectors), np.array(x_solutions)

# Generate training data
num_samples = 1000
b_train, x_train = generate_training_data(num_samples)


/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_isolve/iterative.py:418: RuntimeWarning: overflow encountered in add
  x += alpha*p
/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_isolve/iterative.py:410: RuntimeWarning: overflow encountered in multiply
  p *= beta


In [5]:
# Train the model
history = model.fit(b_train, x_train, epochs=50, batch_size=32)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: nan
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: nan
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: nan
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: nan
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: nan
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: nan
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: nan
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: nan
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: nan
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan
Epoch 13/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: nan
Epoch 14/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan
Epoch 15/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: nan
Epoch 16/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/st

In [6]:
# Evaluate the surrogate model's performance
def evaluate_surrogate_model(b):
    # Use the surrogate model to predict the solution
    x_pred = model.predict(b.reshape(1, -1))  # Reshape for batch prediction
    return x_pred

# Evaluate on a test matrix
A_test = sparse_random(matrix_size, matrix_size, density=density, format='csr', dtype=np.float32)
A_test = A_test @ A_test.T
b_test = np.random.rand(matrix_size).astype(np.float32)

# Traditional PCG solution
x_pcg_test, _ = cg(A_test, b_test)

# Surrogate model solution
x_pred_test = evaluate_surrogate_model(b_test)

# 7. Calculate and print performance improvement
start_time_pcg = time.time()
x_pcg_test, _ = cg(A_test, b_test)
end_time_pcg = time.time()
pcg_time = end_time_pcg - start_time_pcg

start_time_surrogate = time.time()
x_pred_test = evaluate_surrogate_model(b_test)
end_time_surrogate = time.time()
surrogate_time = end_time_surrogate - start_time_surrogate

print(f"PCG solver time: {pcg_time} seconds")
print(f"Surrogate model time: {surrogate_time} seconds")
print(f"Performance improvement: {pcg_time / surrogate_time}x")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
PCG solver time: 0.9618024826049805 seconds
Surrogate model time: 0.06624126434326172 seconds
Performance improvement: 14.51968787342173x
